In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp, str_diff
from t2.utils import get_parser, join_sai, to_sai_str

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
params.my_device='cuda'

print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

modules = build_transformer(env, params)
trainer = RealtimeTrainer(modules, env, params)


INFO - 01/08/23 18:24:07 - 0:00:00 - ============ Initialized logger ============
INFO - 01/08/23 18:24:07 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-657dd551-23fc-4593-8171-0748d559e9dc.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1, my_device='cuda')
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank  

INFO - 01/08/23 18:24:07 - 0:00:00 - Found 370 parameters in model.
INFO - 01/08/23 18:24:08 - 0:00:01 - Optimizers: model


In [6]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAEKAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAEKAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'

print(trainer.single_act_detailed(A,A))

[37, 37, 37, 37, 35, 37, 37, 37, 11, 37, 37, 37, 37, 33, 26, 37, 37, 37, 33, 13, 3, 2, 13, 9, 26, 33, 13, 33, 37, 26, 33, 35, 5, 35, 2, 33, 35, 26, 5, 26, 26, 33, 33, 34, 6, 33, 37, 35, 33, 37, 5, 13, 33, 35, 35, 33, 26, 2, 37, 37, 37, 37, 26, 35, 3, 37, 35, 4, 33, 1, 37, 35, 33, 33, 26, 33, 2, 4, 29, 35, 33, 33, 35, 35, 1, 33, 33, 33, 35, 13, 35, 37, 33, 33, 26, 33, 35, 33, 35, 2, 34, 33, 37, 33, 33, 33, 33, 26, 11, 26, 33, 37, 5, 35, 33, 6, 2, 37, 35, 37, 35, 37, 33, 35, 37, 37, 33, 4, 33, 8, 2, 37, 33, 8, 33]


In [3]:
c = join_sai('123')
b = join_sai('abc')

for l,m in zip(c,b):
    print(l,m)

1 a
   
2 b
   
3 c


In [4]:
start_time = time.time()

losses = []
iterations_number = 10

for it in range(iterations_number):
    for trainer in trainers:
            for _ in range(params.batch_size):
                loss = trainer.learn(a, a)

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number/len(trainers)

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


NameError: name 'trainers' is not defined

In [ ]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()


In [ ]:
print(act(A))

In [ ]:
print(act(B))

In [ ]:
import torch.multiprocessing as mp

In [ ]:
mp.set_start_method('spawn', force=True)

In [ ]:
def m():
    while True:
        print(1)

In [ ]:
p = mp.Process(target=m, args=())

p.start()